# Convert Raw Data -> Excel Files

This IPython Notebook is to transform the storm raw data ('raw_data.txt') into excel files, and categorize them by storm name and years from 2000 onwards.

Data lines for each cyclone is invalid if:
- Wind speed is 0
- The year is not between 2000 & 2022

In [15]:
import pandas as pd
import os

In [16]:
# define helper function
# these create the excel files for each year
def addToExcel(nameList, avgList, dataDict, year, totalList, countTotal):
    dfList = list()
    for i in dataDict.keys():
        dataFrame = pd.DataFrame()
        dataList = dataDict[i]
        for j in range(len(dataList)):
            dataFrame = pd.concat([dataFrame,
                                    pd.DataFrame([dataList[j]],
                                    columns=['date',
                                            'time',
                                            'description',
                                            'lattitude (degree)',
                                            'longitude (degree)',
                                            'pressure (hPa)',
                                            'wind speed (knot)'])],
                                    ignore_index = True)
        dfList.append(dataFrame)
    
    if year < 10:
        filename = 'Excel_Data/Cyclone200' + str(year) + '.xlsx'
    else:
        filename = 'Excel_Data/Cyclone20' + str(year) + '.xlsx'
            
    with pd.ExcelWriter(filename) as writer:
        for i in range(len(nameList)):
            dfList[i].to_excel(writer, sheet_name=nameList[i])
                    
        dataFrame = pd.DataFrame()   
        for i in range(len(avgList)):
            dataFrame = pd.concat([dataFrame,
                                    pd.DataFrame([[nameList[i], avgList[i][0], avgList[i][1], avgList[i][2]]], 
                                    index=['one'], columns=['storm code & name',
                                                            'average latitude',
                                                            'average longitude',
                                                            'average wind'])],
                                    ignore_index = True)
                    
        dataFrame.to_excel(writer, sheet_name='AVERAGE DATA')
        totalList.append(countTotal)

In [17]:
# these descriptions are from the website https://www.jma.go.jp/jma/jma-eng/jma-center/rsmc-hp-pub-eg/Besttracks/e_format_bst.html
description = ["Tropical Depression (TD)",
               "Tropical Storm (TS)",
               "Severe Tropical Storm (STS)",
               "Typhoon (TY)",
               "Extra-tropical Cyclone (L)",
               "Just entering into the responsible area of RSMC Tokyo-Typhoon Center",
               "", # some category description is blank
               "Tropical Cyclone of TS intensity or higher"]

nameList, avgList, totalList = ([] for i in range(3))
dataDict = dict()
countTotal = prevYear = maxLat = maxLong = 0
minLat = minLong = 200

path = "Excel_Data"
isExist = os.path.exists(path)
if not isExist:
   os.makedirs(path)

with open("raw_data.txt") as f:
    for line in f:
        year = int(line[6:8])
        numLine = int(line[12:15])

        # since the year data is 2 digit numbers, year 1950 will be 50
        # and year 2000 will be 0
        if year > 22:
            for i in range(numLine):
                next(f)
            continue

        # if the line is a different year, it means the previous year has ended
        # and all data from previous year is put into its own excel file
        if prevYear != year and len(nameList) != 0:
            addToExcel(nameList, avgList, dataDict, prevYear, totalList, countTotal)
            nameList, avgList = ([] for i in range(2))
            dataDict = dict()
            countTotal = 0
            prevYear = year
        
        avglat = avglong = avgwind = counter = 0
        numberID = line[6:10]

        for i in range(numLine):
            newLine = f.readline()
            wind = int(newLine[33:36])
            
            if wind == 0:
                continue
                
            date = str(newLine[0:6])
            time = str(newLine[6:8])           
            grade = int(newLine[13:14])
            lat = int(newLine[15:18])/10
            long = int(newLine[19:23])/10
            pressure = int(newLine[24:28])
            
            avglat += lat
            avglong += long
            avgwind += wind
            counter += 1

            if numberID not in dataDict.keys():
                dataDict[numberID] = list()

            dataDict[numberID].append([date, time, str(grade) + " - " + description[grade - 2], lat, long, pressure, wind])
        
        avglat = round(avglat/counter, 2)
        avglong = round(avglong/counter, 2)
        avgwind = round(avgwind/counter, 2)
                    
        name = line[29:50].strip()
        sname = numberID + ' - ' + name
        nameList.append(sname)

        avgList.append([avglat, avglong, avgwind])
        countTotal += 1

# the loop will end with 1 less excel file for the year 2022
# thus if the year 2022 has some storm data, add it to an Excel file
if len(nameList) != 0:
    addToExcel(nameList, avgList, dataDict, year, totalList, countTotal)

In [14]:
# create Overall Stats Excel file
year = 2000
dataFrame = pd.DataFrame()
for i in range(len(totalList)):
    dataFrame = pd.concat([dataFrame,
                            pd.DataFrame([[year+i, totalList[i]]],
                            columns=['Year', 'Total Storm'])],
                            ignore_index = True)

with pd.ExcelWriter("Excel_Data/CycloneOverallStats.xlsx") as writer:
    dataFrame.to_excel(writer, sheet_name='overall data')

In [ ]:
# OPTIONAL
# run this cell to remove all the excel file and folder

path = 'Excel_Data'
for i in range(2000, 2023):
    os.remove('Excel_Data/Cyclone' + i + '.xlsx')

os.remove("Excel_Data/CycloneOverallStats.xlsx")
os.rmdir(path)